# 🧪 MLflow Lab: Save • Load • Log (Thai/English)


**วัตถุประสงค์ (Objectives)**  
- เรียนรู้การใช้งาน **MLflow Model**: `save_model`, `load_model`, `log_model`  
- เข้าใจความแตกต่างและการใช้งานที่เหมาะสมของ **Save vs Load vs Log**  
- ติดตามการทดลองด้วย **Tracking URI/Experiment/Run** และตรวจสอบผลผ่าน **MLflow UI**  
- (เสริม) ใช้ **autolog**, **signature**, **input_example**, และโหลดโมเดลแบบ **pyfunc**

> **Environment Note:** ให้ติดตั้งแพ็กเกจก่อน (ในเครื่องผู้เรียน):
> ```bash
> pip install mlflow scikit-learn pandas numpy
> # (ตัวเลือก) เปิด UI ในโฟลเดอร์ที่มี mlruns
> mlflow ui --backend-store-uri mlruns --port 5000
> ```

---

## Dataset
ในตัวอย่างนี้ใช้ข้อมูลสังเคราะห์จาก `sklearn.datasets.make_regression` เพื่อความสะดวก (ไม่มี dependency ข้อมูลภายนอก)


In [ ]:

# --- Basic Imports ---
import os, warnings, json, shutil, tempfile, time
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

warnings.filterwarnings('ignore')

print('MLflow version:', mlflow.__version__)


In [ ]:

# --- Create Synthetic Regression Dataset ---
X, y = make_regression(
    n_samples=1500, n_features=20, n_informative=10,
    noise=0.3, random_state=42
)

X = pd.DataFrame(X, columns=[f"f{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=7
)

X_train.shape, X_test.shape



## 1) ตั้งค่า Tracking URI / Experiment
- **Tracking URI**: กำหนดที่เก็บข้อมูล run และ artifacts (ที่นี่ใช้โฟลเดอร์ `mlruns`)
- **Experiment**: ชื่อกลุ่มการทดลองสำหรับรายวิชา/แลป


In [ ]:

# --- Set up local tracking ---
mlruns_dir = os.path.abspath("mlruns")
os.makedirs(mlruns_dir, exist_ok=True)

mlflow.set_tracking_uri(f"file://{mlruns_dir}")
experiment_name = "MLflow-Lab-Models-Save-Load-Log"
mlflow.set_experiment(experiment_name)

print("Tracking URI:", mlflow.get_tracking_uri())
exp = mlflow.get_experiment_by_name(experiment_name)
print("Experiment:", exp)



## 2) Train & Evaluate (Baseline)
- สร้างโมเดล **ElasticNet** (เหมาะกับตัวอย่าง regression)
- คำนวณ metric: **RMSE** และ **R²**


In [ ]:

# --- Train baseline model ---
alpha, l1_ratio = 0.1, 0.5
model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2   = r2_score(y_test, y_pred)

print(f"Baseline | alpha={alpha}, l1_ratio={l1_ratio}")
print("RMSE:", round(rmse, 4))
print("R^2 :", round(r2, 4))



## 3) **Save / Load** (Local Filesystem)
- `mlflow.sklearn.save_model(model, path)` เพื่อบันทึกเป็นไฟล์ภายในเครื่อง
- `mlflow.sklearn.load_model(path)` เพื่อโหลดกลับมาใช้งานใหม่
- ใช้เมื่อยังไม่ต้องการผูกกับ Experiment/Run (เช่น ต้นแบบอย่างรวดเร็ว)


In [ ]:

# --- Save & Load locally ---
local_model_dir = "models_local/elasticnet_model"
# เคลียร์ก่อน
if os.path.exists(local_model_dir):
    shutil.rmtree(local_model_dir)

mlflow.sklearn.save_model(model, local_model_dir)
print("Saved local model dir:", local_model_dir)

loaded_local = mlflow.sklearn.load_model(local_model_dir)
print("Loaded model type:", type(loaded_local))

# Sanity check
y_pred2 = loaded_local.predict(X_test)
rmse2 = mean_squared_error(y_test, y_pred2, squared=False)
print("RMSE (loaded):", round(rmse2, 4))



## 4) **Log Model** กับ MLflow Tracking
- ใช้ `mlflow.start_run()` เพื่อเริ่มต้น run  
- `mlflow.log_params`, `mlflow.log_metrics` และ `mlflow.sklearn.log_model`
- ข้อดี: บริหาร version, เทียบผล, แนบ artifacts, สืบค้นย้อนหลัง


In [ ]:

# --- Log Model to MLflow Tracking ---
with mlflow.start_run(run_name="elasticnet-baseline"):
    # 4.1 log params & metrics
    mlflow.log_params({"alpha": alpha, "l1_ratio": l1_ratio})
    mlflow.log_metrics({"rmse": rmse, "r2": r2})
    
    # 4.2 add signature & input_example
    signature = infer_signature(X_train, model.predict(X_train))
    input_example = X_train.iloc[:5]
    
    # 4.3 log the model (artifacts under "model")
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=input_example,
        registered_model_name=None  # ใช้ Registry ได้ถ้าตั้งค่าฐานข้อมูล/Server
    )
    
    run_info = mlflow.active_run().info
    print("Logged to Run ID:", run_info.run_id)
    print("Artifact URI:", mlflow.get_artifact_uri())



## 5) Load จาก **Run** ที่ Log แล้ว (pyfunc)
- ดึง `run_id` และ path ของโมเดลใน artifacts: `runs:/<run_id>/model`
- ใช้ `mlflow.pyfunc.load_model` เพื่อโหลดแบบ framework-independent (pyfunc)


In [ ]:

# --- Load back the logged model via runs:/ URI ---
# หา run ล่าสุดใน experiment เพื่อง่ายต่อการสาธิต
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(exp.experiment_id, order_by=["attributes.start_time DESC"], max_results=1)
assert len(runs) > 0, "No runs found"
last_run = runs[0]

model_uri = f"runs:/{last_run.info.run_id}/model"
pyfunc_model = mlflow.pyfunc.load_model(model_uri)

# Predict via pyfunc
y_pred3 = pyfunc_model.predict(X_test)
rmse3 = mean_squared_error(y_test, y_pred3, squared=False)
print("Loaded via pyfunc | RMSE:", round(rmse3, 4))
print("Model URI:", model_uri)



## 6) (เสริม) Autolog
- `mlflow.sklearn.autolog()` จะช่วยบันทึก params/metrics/model โดยอัตโนมัติ
- ใช้สะดวกในการทดลองหลายรอบ


In [ ]:

# --- Autolog demo ---
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(run_name="elasticnet-autolog"):
    mdl2 = ElasticNet(alpha=0.2, l1_ratio=0.7, random_state=7)
    mdl2.fit(X_train, y_train)
    y2 = mdl2.predict(X_test)
    rmse_auto = mean_squared_error(y_test, y2, squared=False)
    r2_auto = r2_score(y_test, y2)
    # แม้ไม่ log เอง MLflow จะบันทึกหลายอย่างให้อัตโนมัติ
    print("Autolog RMSE:", round(rmse_auto, 4), "| R^2:", round(r2_auto, 4))



## 7) สรุป (Summary)
- **Save**: เก็บโมเดลลงไฟล์ local → เหมาะกับต้นแบบ/ทดสอบเร็ว ๆ  
- **Load**: โหลดโมเดลกลับมา predict ได้ทันที  
- **Log**: เก็บรวมกับ **Experiment/Run** → เหมาะกับงานจริง, ทำซ้ำได้, เทียบผลได้, แนบ metadata/artifacts  
- **pyfunc**: โหลดโมเดลแบบ agnostic ต่อ framework  
- **autolog**: สะดวก บันทึกหลายอย่างให้โดยอัตโนมัติ

> เปิด UI เพื่อดูผลลัพธ์:
> ```bash
> mlflow ui --backend-store-uri mlruns --port 5000
> # เปิดเบราว์เซอร์ไปที่: http://127.0.0.1:5000
> ```



---

## 🔐 แบบฝึกหัด (Exercises)
1) เปลี่ยนโมเดลเป็น `RandomForestRegressor` แล้วฝึก/ประเมิน/บันทึก (Save + Log) เทียบกับ ElasticNet  
2) เพิ่ม `input_example` ที่ครอบคลุมค่าขอบเขต (min/max) มากขึ้น แล้วลอง `load_model` กลับมา predict  
3) สร้าง 3 รันโดยปรับ `alpha` และ `l1_ratio` ต่างกัน แล้วดู Grid ใน MLflow UI เปรียบเทียบ `rmse`  
4) (เสริม) ลองใช้ `mlflow.models.evaluate` เพื่อ log metrics และตารางผลลัพธ์เพิ่มเติม (ถ้ามีเวอร์ชันรองรับ)  
5) อธิบายสถานการณ์จริงที่ควรใช้ Save vs Log (เขียนสั้น ๆ 4-5 บรรทัด)

---

**หมายเหตุสำหรับผู้สอน**  
- หากต้องการ Model Registry: ตั้งค่า Tracking Server + Backend DB (เช่น SQLite/MySQL/Postgres) แล้วใช้ `registered_model_name` ตอน `log_model`  
- หากต้องการ Serve เป็น REST: ใช้ `mlflow models serve -m runs:/<run_id>/model -p 8000` (นอกเหนือขอบเขตแลปนี้)
